In [11]:
from enum import Enum, auto
from dataclasses import fields, dataclass, field
import ipywidgets as wgt
from IPython.display import display
from pathlib import Path
import json
from typing import Any
from datetime import datetime
import io
from traceback import format_exc

import numpy as np

In [18]:
from libs.optimizers.algorithms.genetic.operators.fixers import *
from libs.solution.initial_solution_creators.heuristic import *
from libs.solution.initial_solution_creators.random import *
from libs.utils.matrix import extend_cost_mx

## TSP

In [13]:
RND_SEED = 0
rng = np.random.default_rng(seed=RND_SEED)

In [19]:
@dataclass
class CbDataStore:
    data: dict[str, Any] = field(default_factory=dict)
    io_buff: io.StringIO = field(default_factory=io.StringIO)


class ExpT(Enum):
    TSP = auto()
    VRP = auto()
    VRPP = auto()
    IRP = auto()
    UNSPECIFIED = auto()

In [22]:
def generate_initial_population(
    size: int,
    cost_mx: np.ndarray,
    exp_t: ExpT,
    drone_n: int,
    vehicle_volume: float,
):
    quantity_lists = None
    if exp_t == ExpT.TSP:
        drone_n = 1
        vx_seqs = [
            create_tsp_sol_rand(cost_mx, initial_vx, rng)[0] for _ in range(size)
        ]
    else:
        ext_cost_mx = extend_cost_mx(cost_mx, copy_n=drone_n, to_copy_ix=0)
        ini_and_dummy_vxs = {*range(drone_n)}
        if exp_t in [ExpT.VRP, ExpT.VRPP]:
            vx_seqs = [
                create_vrp_sol_rand(
                    ext_cost_mx,
                    initial_ix=0,
                    rng=rng,
                    ini_and_dummy_vxs=ini_and_dummy_vxs,
                )[0]
                for _ in range(size)
            ]
        elif exp_t == ExpT.IRP:
            vx_seqs, quantity_lists = zip(
                *(
                    create_irp_sol_rand(
                        ext_cost_mx,
                        initial_ix=0,
                        rng=rng,
                        ini_and_dummy_vxs=ini_and_dummy_vxs,
                        vehicle_volume=vehicle_volume,
                    )[:-1]
                )
            )
        else:
            raise ValueError(f"`exp_t` not handlable: {exp_t}")
    if quantity_lists is None:
        quantity_lists = [None for _ in vx_seqs]
    return {
        "exp_t": exp_t,
        "drone_n": drone_n,
        "ini_and_nummy_vxs": list(ini_and_dummy_vxs),
        "initial_vx": 0,
        "population": [
            {"vx_seq": vs, "quantities": q} for vs, q in zip(vx_seqs, quantity_lists)
        ],
    }

In [24]:
ENV_LOAD_DIR = Path("./data/environments")
gen_btn = wgt.Button(description="generate missing populations")
data_store = CbDataStore()


def gen_pop_cb(_):
    try:
        if not ENV_LOAD_DIR.exists():
            raise FileNotFoundError(str(load_path))
        for fpath in (p for p in ENV_LOAD_DIR.iterdir() if p.is_file()):
            with fpath.open("r") as f:
                data = json.load(f)
            gen_set = data["generation_setup"]
            for k in (
                "coords",
                "distance",
                "distance_disabled",
                "wind",
                "effective_speed",
                "travel_time",
            ):
                data[k] = np.array(data[k])
            loaded_data[0] = data
    except Exception as e:
        data_store.io_buff.write(f"Exception: {type(e)} - {e}\n{format_exc()}")


load_data_btn.on_click(gen_pop_cb)

Text(value='', description='file to load', placeholder='type json filename')

Button(description='load data', style=ButtonStyle())

In [4]:
# class CreatorType(Enum):
#     HEURISTIC = auto()
#     RANDOM = auto()


# def generate_initial_population(
#     size: int,
#     cost_mx: np.ndarray,
#     prob_of_swap: float,
#     creator_t: CreatorType,
#     initial_vx: int,
# ):
#     """
#     If `creator_t` is `HEURISTIC`, generates one solution and its mutated copies.
#     If `creator_t` is `RANDOM`, generates `n` random solutions.
#     """
#     if creator_t == CreatorType.HEURISTIC:
#         initial_solution = create_tsp_solution_nearest_neighbour(cost_mx, initial_vx)
#         mutated_solutions = (
#             mutate_swap(initial_solution, p=prob_of_swap, rng=rng)[0]
#             for _ in range(size - 1)
#         )
#         return [initial_solution, *mutated_solutions]

#     return [create_tsp_solution_random(cost_mx, initial_vx, rng) for _ in range(size)]

In [5]:
# ENV_LOAD_DIR = Path("./data/environments")
# loaded_data: list[dict[str, Any]] = [{}]
# file_to_load_name_wgt = wgt.Text(
#     description="file to load", placeholder="type json filename"
# )
# load_data_btn = wgt.Button(description="load data")

# str_buff = io.StringIO()


# def load_data_cb(_):
#     try:
#         load_path = ENV_LOAD_DIR / file_to_load_name_wgt.value
#         if not load_path.exists():
#             raise FileNotFoundError(str(load_path))
#         with load_path.open("r") as f:
#             data = json.load(f)
#         for k in (
#             "coords",
#             "distance",
#             "distance_disabled",
#             "wind",
#             "effective_speed",
#             "travel_time",
#         ):
#             data[k] = np.array(data[k])
#         loaded_data[0] = data
#     except Exception as e:
#         str_buff.write(f"Exception: {type(e)} - {e}\n{format_exc()}")


# load_data_btn.on_click(load_data_cb)
# display(file_to_load_name_wgt)
# display(load_data_btn)

Text(value='', description='file to load', placeholder='type json filename')

Button(description='load data', style=ButtonStyle())

In [ ]:
str_buff.seek(0)
print(str_buff.read())

In [6]:
SAVE_DIR = Path("./data/populations/tsp")
SAVE_PATH_TIME_FMT = "%Y-%m-%d_%H-%M-%S"

cost_mx = loaded_data[0]["travel_time"]

generated_population = [[]]
initial_costs = [[]]

widgets = {
    "initial_vx": wgt.BoundedIntText(value=0, min=0, max=cost_mx.shape[0] - 1),
    "size": wgt.BoundedIntText(value=2, min=2, max=100, step=2),
    "prob_of_swap": wgt.BoundedFloatText(value=0.1, min=0, max=1, step=0.05),
    "creator_t": wgt.Dropdown(
        value=CreatorType.RANDOM, options=[(e.name, e) for e in CreatorType]
    ),
}
LABEL_WIDTH = "150px"
descriptions = {
    "initial_vx": "initial vertex",
    "size": "population size",
    "prob_of_swap": "prob of vertex swap",
    "creator_t": "creation method",
}
pop_gen_wgts = wgt.VBox(
    [
        wgt.HBox(
            children=[
                wgt.Label(  # type: ignore
                    descriptions[widget_key], layout=wgt.Layout(width=LABEL_WIDTH)  # type: ignore
                ),
                widgets[widget_key],
            ]
        )
        for widget_key in widgets
    ]
)

str_buff = io.StringIO()


def create_population_cb(_):
    """
    Generates population and saves it.
    """
    try:
        widget_vals = {k: w.value for k, w in widgets.items()}
        initial_vx = widget_vals["initial_vx"]
        new_pop = generate_initial_population(cost_mx=cost_mx, **widget_vals)
        fixed_pop_and_fix_results = [fix_tsp(c, cost_mx, initial_vx) for c in new_pop]
        fixed_pop = [x[0] for x in fixed_pop_and_fix_results]
        fix_results = [x[1] for x in fixed_pop_and_fix_results]
        fix_results_parsed = [
            {"no_of_errors": fr.no_of_errors, "fix_status": fr.fix_status.name}
            for fr in fix_results
        ]
        fixed_pop_costs = [
            calculate_total_cost(cost_tsp_gen(s, cost_mx, initial_vx))
            for s in fixed_pop
        ]
        generated_population[0] = fixed_pop
        initial_costs[0] = fixed_pop_costs
        now_str = datetime.now().strftime(SAVE_PATH_TIME_FMT)
        save_path = SAVE_DIR / f"population_tsp_{now_str}.json"
        with save_path.open("w") as f:
            json.dump(
                {
                    "population": fixed_pop,
                    "costs": fixed_pop_costs,
                    "fix_results": fix_results_parsed,
                },
                f,
            )
    except Exception as e:
        str_buff.write(f"Exception: {type(e)} - {e}\n{format_exc()}")


generate_btn = wgt.Button(description="generate population")
generate_btn.on_click(create_population_cb)

display(pop_gen_wgts)
display(generate_btn)

KeyError: 'travel_time'

In [31]:
str_buff.seek(0)
print(str_buff.read())

[]



In [43]:
generated_population

[[[0, 7, 8, 5, 6, 4, 3, 1, 2],
  [0, 2, 4, 1, 6, 8, 3, 7, 5],
  [0, 3, 6, 7, 2, 8, 4, 1, 5],
  [0, 5, 3, 7, 6, 2, 1, 8, 4],
  [0, 3, 6, 8, 7, 5, 1, 2, 4],
  [0, 4, 1, 6, 5, 3, 8, 7, 2],
  [0, 3, 8, 7, 2, 4, 5, 1, 6],
  [0, 1, 2, 5, 8, 3, 6, 7, 4],
  [0, 8, 6, 7, 4, 1, 2, 5, 3],
  [0, 6, 7, 2, 4, 5, 3, 1, 8]]]

## TODOs

- MTSP - jak wyżej
- MVMTSP - jak wyżej, inne mutatory, liczyć koszt na podst. wizyt w wierzchołkach
- IRP - inne mutatory, koszt na podst. zostawianego towaru, który odnawia się
  po wizycie w depocie (optymalizacja - powrót do depota najkrótszą drogą (Dijkstra)
  po zużyciu towaru)
- jupyterowy kreator configów
- scheduler dla eksperymentów